# 大语言模型预训练之全量参数微调

在当今的自然语言处理领域，全量参数微调（Fine-tuning）已成为释放大型预训练语言模型潜力的关键技术手段。然而，随着模型规模的扩大，这一过程对计算资源的要求也急剧上升。

在本次实践中，我们选择了一个拥有约8亿参数的模型进行全量参数微调。这一规模级别的模型，通常需要大约18GB的显存资源。尽管这在现代硬件上是可行的，但当模型规模扩大到80亿参数时，所需的显存将飙升至180GB，这种级别的资源消耗通常只有资金雄厚的企业才能承担。

进一步地，如果模型的规模达到惊人的800亿参数，那么所需的显存将达到庞大的1800GB。如此巨大的资源需求，即便是对于许多公司而言，也是一项极具挑战的任务，往往超出了他们的能力范围。因此，尽管全量参数微调在技术上可行，但在实际的应用和研究中，由于资源的限制，这样的实践相对较少。

鉴于此，大多数实际应用场景倾向于采用部分微调（Partial Fine-tuning）或迁移学习，这样既可以利用预训练模型中的知识，又可以针对性地调整模型以适应特定任务，而无需动用如全量微调那般庞大的计算资源。这种方法更加经济高效，同时也能产生令人满意的结果。

## 步骤1：导入相关包

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## 步骤2：加载数据集

In [ ]:
ds = Dataset.load_from_disk("data/alpaca_data_zh/")
ds

In [ ]:
ds[:3]

## 步骤3：数据预处理

### 1）获取分词器

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-800m-zh")
tokenizer

### 2）定义数据处理函数

In [ ]:
def process_func(example):
    MAX_LENGTH = 256  # 定义最大长度为256
    input_ids, attention_mask, labels = [], [], []  # 初始化输入ID、注意力掩码和标签列表
    # 对指令和输入进行编码，返回输入ID和注意力掩码
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    # 对输出进行编码，返回输出ID和注意力掩码
    response = tokenizer(example["output"] + tokenizer.eos_token)
    # 将指令和回应的输入ID合并
    input_ids = instruction["input_ids"] + response["input_ids"]
    # 将指令和回应的注意力掩码合并
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    # 标签列表的前半部分是指令的长度个-100（表示这些位置的标签是被忽略的），后半部分是回应的输入ID
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    # 如果输入ID的长度大于最大长度，将其截断为最大长度
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    # 返回一个包含输入ID列表、注意力掩码列表和标签列表的字典
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

### 3）对数据进行预处理

In [ ]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

### 4）数据格式检查

In [ ]:
#检查一下数据格式，知识部分是否符合我们的需求
tokenizer.decode(tokenized_ds[1]["input_ids"])

In [ ]:
## 检查一下数据格式，目标值是否符合我们的需求
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

## 步骤4：创建模型

In [ ]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-800m-zh")

## 步骤5：配置训练参数

In [ ]:
args = TrainingArguments(
    output_dir="/root/autodl-tmp/tuningdata/boomtuning",# 指定模型训练结果的输出目录。
    per_device_train_batch_size=4,  # 指定每个设备（如GPU）上的批次大小
    gradient_accumulation_steps=8,  # 指定梯度累积步数。在本例子中，每8个步骤进行一次梯度更新。
    logging_steps=10, #指定日志记录的频率。在本例子中，每10个步骤记录一次日志
    num_train_epochs=1 #指定训练的总轮数。在本例子中，训练将进行1轮, 实际使用是会是多轮
)

## 步骤6：创建训练器
data_collator在训练机器学习模型时，有以下作用：

1）数据转换：data_collator负责将输入的特征数据转换成统一形状和格式的张量(tensor)，这是为了便于模型进行统一的处理。

2）批量处理：它能够将多个数据样本整合成一个小批次(batch)的数据，这有助于提高模型训练的效率。

3）填充(padding)：在文本处理中，不同大小的输入需要被填充或截断到相同的长度以形成统一的形状，这对于很多自然语言处理模型来说是必要的，而DataCollatorWithPadding就是执行这一操作的常用collator。

> 综上所述，data_collator是连接数据处理与模型训练之间的重要桥梁，确保了数据的有效整理和组织，以便模型可以高效地从中学习。

In [ ]:
trainer = Trainer(
    model=model,#指定训练模型
    args=args, #指定训练参数
    train_dataset=tokenized_ds, #指定数据集
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True) #指定数据收集器。其中tokenizer是分词器，padding=True表示对输入进行填充以保持批次大小一致。
)

## 步骤7：模型训练

In [ ]:
trainer.train()

## 步骤8：模型推理

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)
ipt = "Human: {}\n{}".format("如何写好一个简历？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True, )